<a href="https://colab.research.google.com/github/TarnishingColors/web_mining_hate-speech-detection/blob/main/web_mining_project_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving d.csv to d.csv
Saving d1.csv to d1.csv


In [2]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['d.csv']), encoding="utf-8", encoding_errors='ignore')

In [3]:
df1 = pd.read_csv(io.BytesIO(uploaded['d1.csv']), encoding="utf-8", encoding_errors='ignore')

In [4]:
d1_df = pd.DataFrame()
d1_df['class'] = df1['label']
d1_df['tweet'] = df1['tweet']
d1_df

,class,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
31957,0,ate @user isz that youuu?ðððððð...
31958,0,to see nina turner on the airwaves trying to...
31959,0,listening to sad songs on a monday morning otw...
31960,1,"@user #sikh #temple vandalised in in #calgary,..."


In [5]:
!pip install emot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.6 MB/s eta 0:00:00


In [6]:
import re
import pickle
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS

In [7]:
#custom preprocessing for this specific dataset
df.drop(['count', 'offensive_language','neither','hate_speech'], axis = 1, inplace = True)
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df_first = df["class"].transform(lambda x: x.replace([1,2], "XX"))
df["class"] = df_first.transform(lambda x: x.replace(0,1).replace("XX",0)) #hate-speach = 1 in class everitying else  = 0
display(df)

,class,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,0,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,0,"you've gone and broke the wrong heart baby, an..."
24780,0,young buck wanna eat!!.. dat nigguh like I ain...
24781,0,youu got wild bitches tellin you lies


In [8]:
df = df.append(d1_df)
df

<ipython-input-8-e0a1b7225b8f>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d1_df)


,class,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
31957,0,ate @user isz that youuu?ðððððð...
31958,0,to see nina turner on the airwaves trying to...
31959,0,listening to sad songs on a monday morning otw...
31960,1,"@user #sikh #temple vandalised in in #calgary,..."


In [9]:
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.9 MB/s eta 0:00:00


In [29]:
import html
import re

import nltk
import pandas as pd
import torch
from emot.emo_unicode import UNICODE_EMOJI  # For emojis
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.base import TransformerMixin, BaseEstimator
from transformers import BertTokenizer, BertModel

from toolz import pipe
from functools import reduce


def compose(*functions):
    return reduce(lambda f, g: lambda x: f(g(x)), functions[::-1])


# Specify the pre-trained model name.
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'  # uncased means this tokenizer will also lower-case automatically

# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)


# class for preprocessing
class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, func: callable):
        self.func = func

    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        return self

    def transform(self, df: pd.DataFrame, y=None, **transform_params):
        _df = df.copy()
        _df["tweet"] = _df["tweet"].transform(lambda x: self.func(x))
        return _df


# Lemmatization

nltk.download('stopwords')
nltk.download('punkt')

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


class PrepFuncs:
    re_username = r"@\w+"
    re_exclamation = ""
    re_link = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
                  '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    re_all_words = "[^a-z ]+"

    @classmethod
    def get_wordnet_pos(self, treebank_tag: str):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN

    # Functions for extracting 'rt_flag' feature
    @classmethod
    def annotate_rt(text: str):
        return 1 if len(text.split()) > 0 and text.split()[0] == 'rt' else 0

    @classmethod
    def delete_rt_info(text: str):
        if len(text) > 0:
            return text if text.split()[0] != 'rt' else " ".join(text.split()[1:])
        return text

    # Function for converting emojis into word
    @classmethod
    def convert_emojis(self, text: str):
        text = html.unescape(text)
        for emot in UNICODE_EMOJI:
            text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",", "").replace(":", "").split()))
        return text

    @classmethod
    def lemmatize(self, text: str):
        lemmatized = [
            lemmatizer.lemmatize(word, self.get_wordnet_pos(pos_tag)) 
            for (word, pos_tag) in nltk.pos_tag(nltk.word_tokenize(text))
            ]
        return " ".join(lemmatized)

    @classmethod
    def remove_stopwords(self, tweet: str):
        tokens_without_sw = " ".join(
            [word for word in word_tokenize(tweet) if not word in stop_words]
        )
        return tokens_without_sw
    
    @classmethod
    def delete_usernames(self, text: str):
        return re.sub(self.re_username, "username", text)
    
    @classmethod
    def delete_links(self, text: str):
        return re.sub(self.re_link, "link", text)
    
    @classmethod
    def delete_special_symbols(self, text: str):
        return re.sub(self.re_all_words, " ", text)
    
    @classmethod
    def to_lower(self, text: str):
        return text.lower()

    @staticmethod
    def preprocessing():
        return compose(
            PrepFuncs.delete_usernames,
            PrepFuncs.delete_links,
            PrepFuncs.convert_emojis,
            PrepFuncs.to_lower,
            PrepFuncs.delete_special_symbols,
            PrepFuncs.lemmatize,
            PrepFuncs.remove_stopwords
        )


def preprocessing(text: str):
    return PrepFuncs.preprocessing()(text)


class BertVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, tokenizer: callable, max_len: int):
        self.model = BertModel.from_pretrained('bert-base-uncased')
        self.tokenizer = tokenizer

        self.config = {
            'add_special_tokens': True,
            'padding': 'max_length',
            'max_length': max_len,
            'truncation': True,
            'pad_to_max_length': True,
            'return_attention_mask': False,
            'return_tensors': 'pt',
        }

    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        return self

    def transform(self, df: pd.DataFrame, y=None, **transform_params):
        with torch.no_grad():
            _df = df \
                .apply(lambda x: self.tokenizer(x['tweet'], **self.config)['input_ids'], axis=1) \
                .apply(lambda x: pd.Series(self.model(x)[0].flatten()))

        return _df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
df.iloc[0]

class                                                    0
tweet    !!! RT @mayasolovely: As a woman you shouldn't...
Name: 0, dtype: object

In [24]:
preprocessing("!!! RT @mayasolovely: As a woman you shouldn't...")

!!! RT @mayasolovely: As a woman you shouldn't...
!!! RT @mayasolovely: As a woman you shouldn't...
!!! RT username: As a woman you shouldn't...
!!! RT username: As a woman you shouldn't...
!!! RT username: As a woman you shouldn't...
!!! rt username: as a woman you shouldn't...
  rt username  as a woman you shouldn t 
rt username a a woman you shouldn t
rt username woman


'rt username woman'

In [30]:
from sklearn.pipeline import Pipeline


sliced_df = df.copy()[:500]

pipeline = Pipeline([
    ("preprocessing", DataCleaner(preprocessing)),
])

transformed_df = pipeline.fit_transform(sliced_df)
transformed_df

,class,tweet
0,0,rt username woman complain clean house man alw...
1,0,rt username boy dat cold tyga dwn bad cuffin d...
2,0,rt username dawg rt username ever fuck bitch s...
3,0,rt username username look like tranny
4,0,rt username shit hear might true might faker b...
...,...,...
495,0,funky honkey username shout username tonight c...
496,0,compare kfc pussy username
497,0,download new personality order new face bitch ...
498,0,feelin like malcolm middle bitch surround miss em


In [45]:
# BERT pipeline
from sklearn.pipeline import Pipeline


sliced_df = df.copy()[:500]

pipeline = Pipeline([
    ("preprocessing", DataCleaner(preprocessing)),
    ("vectorizing", BertVectorizer(tokenizer, 32))
])

transformed_df = pipeline.fit_transform(sliced_df)
transformed_df

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,0,1,2,3,4,5,6,7,8,9,...,24566,24567,24568,24569,24570,24571,24572,24573,24574,24575
0,-0.247633,0.262839,0.373891,0.127794,-0.057450,0.000630,0.431298,-0.153828,-0.002902,-0.076881,...,-0.117364,0.076719,-0.130772,0.060906,0.613288,-0.761720,-0.024896,-0.487892,0.248999,-1.050962
1,-0.462687,0.092296,0.241317,0.177272,-0.220461,-0.097000,0.402557,0.097665,-0.094378,-0.068383,...,-0.263822,0.327823,0.007033,0.104683,0.768214,-0.959925,0.038983,-0.592973,0.345280,-1.213694
2,-0.228533,0.270534,0.426521,0.247550,-0.106371,-0.128783,0.358361,0.100560,0.051267,0.046411,...,-0.151044,0.072961,-0.151868,0.129745,0.632372,-0.695994,-0.107353,-0.514699,0.232668,-0.998355
3,-0.231747,0.122291,0.338131,0.225593,-0.237738,-0.332537,0.475940,0.062612,-0.013484,0.047540,...,-0.117105,0.008060,-0.225146,-0.008859,0.503046,-0.510885,-0.085963,-0.452896,0.184116,-0.857576
4,0.014354,0.302597,0.388331,0.276414,-0.163741,-0.247846,0.531410,0.075458,0.132352,-0.157904,...,-0.127670,0.057154,-0.188779,0.118632,0.533638,-0.718177,-0.115379,-0.505064,0.162552,-1.034123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.419492,0.090035,0.408407,0.216683,-0.234387,-0.299037,0.386116,-0.021153,0.041413,-0.206114,...,-0.100577,0.148721,-0.093687,0.011891,0.609602,-0.756266,-0.085257,-0.478104,0.220872,-0.955893
496,-0.646306,-0.063289,0.206763,0.121152,-0.201566,-0.120941,0.420506,-0.137670,-0.092997,0.060009,...,-0.097133,0.038380,-0.164404,-0.043042,0.537096,-0.550906,0.011793,-0.459521,0.183238,-0.932019
497,-0.396464,-0.073714,0.212740,0.177693,-0.461200,0.117577,0.405961,-0.174486,-0.221864,-0.345992,...,-0.130732,0.283193,0.203598,0.161469,0.860813,-1.034812,0.167028,-0.556451,0.251271,-1.427649
498,-0.395544,0.052431,0.346651,0.273494,-0.120987,-0.023858,0.439511,0.127272,0.052679,-0.054985,...,-0.188073,0.261162,-0.161410,0.007722,0.606285,-0.734997,-0.023007,-0.552948,0.192827,-1.017917


In [6]:
import html


# Functions for extracting 'rt_flag' feature
def annotate_rt(text):
    return 1 if len(text.split()) > 0 and text.split()[0] == 'rt' else 0


def delete_rt_info(text):
    if len(text) > 0:
        return text if text.split()[0] != 'rt' else " ".join(text.split()[1:])
    return text


# Function for converting emojis into word
def convert_emojis(text):
    text = html.unescape(text)
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text

In [7]:
# Lemmatization
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()


def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


def lemmatize(text):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in tagged]
    return " ".join(lemmatized)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


stop_words = set(stopwords.words('english'))


def remove_stopwords(tweet):
  tokens_tweet = word_tokenize(tweet)
  tokens_without_sw = " ".join([word for word in tokens_tweet if not word in stop_words])

  return tokens_without_sw

In [9]:
import re

# class for preprocessing
class DataframePreprocessor():
    def __init__(self, func: callable):
        self.func = func
    
    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        return self
    
    def transform(self, df: pd.DataFrame, y=None, **transform_params):
        df["tweet"], df["rt_flag"] = zip(*df["tweet"].transform(lambda x: self.func(x)))
        return df


# also adds a new column 'rt_flag'
def preprocessing(tweet):
    re_username = '@[\w\-]+'
    re_exclamation = ""
    re_link = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
          '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    re_all_words = "[^a-z ]+"

    tweet = re.sub(re_username,"", tweet)
    tweet = re.sub(re_link,"link", tweet)
    tweet = tweet.lower()
    tweet = convert_emojis(tweet)
    tweet = re.sub(re_all_words," ",tweet)
    tweet = lemmatize(tweet)
    tweet = remove_stopwords(tweet)
    rt_flag = annotate_rt(tweet)
    tweet = delete_rt_info(tweet)

    return tweet, rt_flag

In [10]:
preprocessing(df.iloc[174]['tweet'])

('wish bae confused face get hoe tho', 0)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer


# class for vectorizing
class TfIdfVectorizer():
    def __init__(self, vectorizer: callable):
        self.vectorizer = vectorizer
    
    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        return self
    
    def transform(self, df: pd.DataFrame, y=None, **transform_params):
        X = self.vectorizer.fit_transform(df["tweet"])
        new_df = pd.DataFrame(X.toarray(), columns=self.vectorizer.get_feature_names_out())
        new_df["rt_flag"] = df["rt_flag"]
        new_df["label"] = df["class"] # 'class' is the name of label in d.csv, so we'll need to standartize all the datasets
        return new_df


vectorizer = TfidfVectorizer(norm='l2', min_df=0.05, max_df=0.5)

In [70]:
from sklearn.pipeline import Pipeline


sliced_df = df.copy()[:20000]

pipeline = Pipeline([
    ("preprocessing", DataframePreprocessor(preprocessing)),
    ("vectorizing", TfIdfVectorizer(vectorizer))
])

transformed_df = pipeline.fit_transform(df)
display(transformed_df)

KeyboardInterrupt: ignored

In [14]:
from transformers import BertTokenizer
# Specify the pre-trained model name.
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased' # uncased means this tokenizer will also lower-case automatically

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Loading BERT tokenizer...


In [29]:
#only for searching a max len sake, not for a final version
pipeline = Pipeline([
    ("preprocessing", DataframePreprocessor(preprocessing)),
])

transformed_df = pipeline.fit_transform(df)
transformed_df['len'] = transformed_df['tweet'].transform(lambda x: len(x.split()))

In [31]:
max(transformed_df['len'])

29

In [81]:
import torch


class BertVectorizer():
    def __init__(self, tokenizer: callable):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attention_masks = []
    
    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        return self
    
    def transform(self, df: pd.DataFrame, y=None, **transform_params):
        for text in df['tweet']:
            encoded_text = tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=64, #should be looked into later on a full dataset, as calculated above 29 is max len on a slice of 500 rows
                pad_to_max_length=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
            self.input_ids.append(encoded_text['input_ids'])
            self.attention_masks.append(encoded_text['attention_mask'])
            
        self.input_ids = torch.cat(self.input_ids, dim=0)
        self.attention_masks = torch.cat(self.attention_masks, dim=0) # if we are going to use sklearn, then it is not needed
        new_df = pd.DataFrame(self.input_ids)
        new_df['label'] = torch.tensor(df['class'].values)
        return new_df

In [96]:
transformed_df.groupby(['label']).count()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
label,,,,,,,,,,,,,,,,,,,,,
0,46800,46800,46800,46800,46800,46800,46800,46800,46800,46800,...,46800,46800,46800,46800,46800,46800,46800,46800,46800,46800
1,3200,3200,3200,3200,3200,3200,3200,3200,3200,3200,...,3200,3200,3200,3200,3200,3200,3200,3200,3200,3200


In [92]:
from sklearn.model_selection import train_test_split

train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    transformed_df[transformed_df.columns.difference(['label'])], transformed_df['label'], test_size=0.2, random_state=42
)
# train_masks, test_masks, _, _ = train_test_split(
#     attention_masks, input_ids, test_size=0.2, random_state=42
# )

In [107]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(class_weight={0:1,1:7})
model.fit(train_inputs, train_labels)

RandomForestClassifier(class_weight={0: 1, 1: 7})

In [108]:
from sklearn.metrics import f1_score, confusion_matrix

predictions = model.predict(test_inputs)
f1 = f1_score(test_labels, predictions)
cm = confusion_matrix(test_labels, predictions)
print("f1:", f1)
print(cm)

f1: 0.2516914749661705
[[9354   22]
 [ 531   93]]
